In [1]:
campaign_id = 1
manager_ethereum_address = '0x5B93FF82faaF241c15997ea3975419DDDd8362c5'

In [2]:
import base64
import mysql.connector as mysql
import datetime
import pandas as pd
import time
import requests

import json
from web3 import Web3 # https://web3py.readthedocs.io/en/stable/contracts.html#contract-functions

#######
## INIT
#######

# load api key
secret = {}
with open('secret.txt') as f:
    lines = f.readlines()
    for line in lines:
        secret[line.split("=")[0]] = line.split("=")[1].replace("\n","")

# init web3
infura_url = secret['INFURAURL1']
web3 = Web3(Web3.HTTPProvider(infura_url))
print(f"Connected to infura: {infura_url}")

# load abi
with open('abi.json') as f:
    abi = json.load(f)

# load bytecode
with open('bytecode.txt', 'r') as file:
    bytecode = file.read().replace('\n', '')

# load contract address
contract_address = secret['CONTRACTADDRESS']

Consider installing rusty-rlp to improve pyrlp performance with a rust based backend


Connected to infura: https://mainnet.infura.io/v3/1070d8486bc64afdb9c730d6f2850d6c


In [3]:
###########################
## GET USER:WALLET MAPPINGS
###########################

# convert rewards pending twitter handles to wallet ids
db = mysql.connect(host=secret['DBHOST'],user=secret['DBUSER'],passwd=secret['DBPASS'],database=secret['DBTABLE'])
cursor = db.cursor()
query = f'SELECT twitter_handle, ethereum_address FROM users;'
cursor.execute(query)
records = cursor.fetchall()
cursor.close()
db.close()
# convert to dataframe
users = []
for record in records:
    users.append(dict(zip(['twitter_handle', 'ethereum_address'], record)))
# convert to dataframe and lowercase handle
users = pd.DataFrame(users)
users['twitter_handle'] = users['twitter_handle'].str.lower()

# create dicts
user_wallets = dict(zip(list(users['twitter_handle']), users['ethereum_address']))
wallets_users = dict(zip(list(users['ethereum_address']), users['twitter_handle']))

# create list
user_list = list(user_wallets.keys())

In [4]:
##################################################
### get all rewards for this campaign as dataframe
##################################################
db = mysql.connect(host=secret['DBHOST'],user=secret['DBUSER'],passwd=secret['DBPASS'],database=secret['DBTABLE'])
cursor = db.cursor()
query = f'SELECT campaign_id, twitter_handle FROM rewards WHERE blockchain_write_time is NULL and manager_ethereum_address = %s and campaign_id = %s;'
values = (manager_ethereum_address, campaign_id)
cursor.execute(query, values)
records_rewards = cursor.fetchall()
cursor.close()
db.close()
# convert to dataframe
rewards = []
for record in records_rewards:
    rewards.append(dict(zip(['campaign_id', 'twitter_handle'], record)))
df = pd.DataFrame(rewards)
print(f"DEGBUG: LEN DF {len(df)}")

DEGBUG: LEN DF 2


In [5]:
###################################################
### GET LIST OF WALLETS TO REWARD FOR THIS CAMPAIGN
###################################################

# list of handles to reward
rewards_pending = list(df[df['campaign_id'] == campaign_id]['twitter_handle'])

# convert from twitter handles to wallets
rewards_pending = [user_wallets[r] for r in rewards_pending]

In [6]:
print(f"{len(rewards_pending)} rewards_pending")
for reward_pending in rewards_pending:
    print(reward_pending, wallets_users[reward_pending])

2 rewards_pending
0x3708045C19c4D0eaB69289F8BC10D4a1fF4EDc6a alxcnwy
0x5B93FF82faaF241c15997ea3975419DDDd8362c5 cooopahtroopa


In [7]:
rewards_pending = ['0x5B93FF82faaF241c15997ea3975419DDDd8362c5']

In [8]:
# get current gas price
headers = {'User-Agent': 'blah',}
response = requests.get('https://ethgasstation.info/json/ethgasAPI.json', headers=headers)
gasPrice = int(response.json()['average']/10)
print(gasPrice)

40


In [9]:
for i, reward_pending in enumerate(rewards_pending):
    print(f"Creating SC tx {i+1}/{len(rewards_pending)} to reward {reward_pending}")

    ###################################
    ### WRITE REWARDS TO SMART CONTRACT
    ###################################
    
    # get gas price from ethgasstation.info
    gasPrice = 25
    try:
        headers = {'User-Agent': 'blah',}
        response = requests.get('https://ethgasstation.info/json/ethgasAPI.json', headers=headers)
        gasPrice = int(response.json()['average']/10)
        print(f"GasPrice fetched from ethgasstation.info {gasPrice}")
    except Exception as e:
        print("ERROR FETCHING GAS PRICE - using default of 25")
        print(e)


    contract = web3.eth.contract(abi=abi, bytecode=bytecode)

    tx = contract.functions.rewardAddresses(manager_ethereum_address, campaign_id, [reward_pending]).buildTransaction(
        {'gas':250000,
         'gasPrice': web3.toWei(gasPrice, 'gwei'),
         'from': secret['ETHBACKENDPUBLIC'],
         'to': contract_address,
         'nonce': web3.eth.getTransactionCount(secret['ETHBACKENDPUBLIC'])
        })

    signed_txn = web3.eth.account.signTransaction(tx, private_key=secret['ETHBACKENDPRIVATE'])
    tx_hash = web3.eth.sendRawTransaction(signed_txn.rawTransaction)
    print(f"TX hash: {tx_hash.hex()}")
    receipt = web3.eth.waitForTransactionReceipt(tx_hash, timeout=3600)
    print(receipt)

    print(f"LOGGING: {tx_hash}, rewarding: {reward_pending}, manager: {manager_ethereum_address}, ")

    if receipt['status'] == 1:
        print(f"SUCCESS writing rewards to SC for campaign {campaign_id} tx: {tx_hash.hex()}")

        # update blockchain_write_time
        db = mysql.connect(host=secret['DBHOST'],user=secret['DBUSER'],passwd=secret['DBPASS'],database=secret['DBTABLE'])
        cursor = db.cursor()
        query = "UPDATE rewards SET blockchain_write_time=%s, blockchain_write_tx_hash=%s, gas_used=%s, blockchain_write_tx_status=%s WHERE campaign_id=%s AND manager_ethereum_address=%s AND twitter_handle=%s;"
        values = (str(datetime.datetime.now()).split('.')[0], str(tx_hash.hex()), receipt['gasUsed'], 1, str(campaign_id), manager_ethereum_address, wallets_users[reward_pending])
        cursor.execute(query, values)
        db.commit()
        print(cursor.rowcount, "record updated")
        cursor.close()
        db.close()  
    else:
        print(f"ERROR FAIL writing rewards to SC for campaign {campaign_id} tx: {tx_hash.hex()}")

Creating SC tx 1/1 to reward 0x5B93FF82faaF241c15997ea3975419DDDd8362c5
GasPrice fetched from ethgasstation.info 39
TX hash: 0x45980bcfb862ee6ec46218ea40473591fd4b2f0f34c0962f577a364aa8c7b060
AttributeDict({'blockHash': HexBytes('0x6ac6204dc161c89ae6cf68d40712362ec560cf89f8503c6dbe725356f9fa0919'), 'blockNumber': 11501558, 'contractAddress': None, 'cumulativeGasUsed': 10846788, 'from': '0xe9540071278Dba4374A42a84B48c5e230328AeF7', 'gasUsed': 160280, 'logs': [AttributeDict({'address': '0xe7b8897D824C78e5C1076023508C43Eace3D2Afc', 'blockHash': HexBytes('0x6ac6204dc161c89ae6cf68d40712362ec560cf89f8503c6dbe725356f9fa0919'), 'blockNumber': 11501558, 'data': '0x0000000000000000000000007d91e637589ec3bb54d8213a9e92dc6e8d12da910000000000000000000000000000000000000000000000000000000000002710', 'logIndex': 267, 'removed': False, 'topics': [HexBytes('0xeeecc2fca78e5b3eaac3c78bc0db4c19d0b8d600a75bbb4911c4a5e458a515ac'), HexBytes('0x0000000000000000000000005b93ff82faaf241c15997ea3975419dddd8362c5'),

# Check confirmation status

In [59]:
##############################################################
### check confirmation status for all unconfirmed transactions
##############################################################

# get all unconfirmed txs
db = mysql.connect(host=secret['DBHOST'],user=secret['DBUSER'],passwd=secret['DBPASS'],database=secret['DBTABLE'])
cursor = db.cursor()
query = f'SELECT blockchain_write_tx_hash FROM rewards WHERE blockchain_write_tx_status = 0;'
cursor.execute(query)
records_txs = cursor.fetchall()
cursor.close()
db.close()
# convert to dataframe
txs = []
for record in records_txs:
    txs.append(record[0])
print(f"DEGBUG: UNCONFIRMED TXS {len(txs)}")

# check tx status and update db if confirmed
for tx in txs:
    receipt = web3.eth.getTransactionReceipt(tx)
    
    if receipt['status'] == 1:
        # update blockchain_write_time
        db = mysql.connect(host=secret['DBHOST'],user=secret['DBUSER'],passwd=secret['DBPASS'],database=secret['DBTABLE'])
        cursor = db.cursor()
        query = "UPDATE rewards SET blockchain_write_time=%s, blockchain_write_tx_status=%s, gas_used=%s WHERE campaign_id=%s AND manager_ethereum_address=%s AND twitter_handle=%s;"
        values = (str(datetime.datetime.now()).split('.')[0], 1, receipt['gasUsed'], str(campaign_id), manager_ethereum_address, wallets_users[reward_pending])
        cursor.execute(query, values)
        db.commit()
        print(cursor.rowcount, "record updated")
        cursor.close()
        db.close() 
    else:
        print(f"ERROR - {tx} receipt status != 1")
        print(receipt)

Rewarding 1/1 0xe9540071278Dba4374A42a84B48c5e230328AeF7
TX hash: 0xb1cee719715c454467106ffc2c6b73c6234b74f7b5486bf84c336fb49b70e618
AttributeDict({'blockHash': HexBytes('0xffb054245778a31764542448d8fb31a865865ad0cadf4a5465ee2f1ac315d557'), 'blockNumber': 22328506, 'contractAddress': None, 'cumulativeGasUsed': 22208, 'from': '0xe9540071278Dba4374A42a84B48c5e230328AeF7', 'gasUsed': 22208, 'logs': [], 'logsBloom': HexBytes('0x00000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000'), 'status': 1, 'to': '0xe7b8897D824C78e5C1076023508C43Eace

# Estimate gas

In [57]:
contract.address = contract_address

In [58]:
contract.address

'0xe7b8897D824C78e5C1076023508C43Eace3D2Afc'

In [59]:
tx = contract.functions.rewardAddresses(ids_manageraddresses[campaign_id], int(campaign_id), rewards_pending).buildTransaction(
    {'gas':3000000, 
     'gasPrice': web3.toWei('23', 'gwei'),
     'from': secret['ETHBACKENDPUBLIC'],
     'to': contract_address,
     'nonce': web3.eth.getTransactionCount(secret['ETHBACKENDPUBLIC'])
    })


In [62]:
tx.estimateGas()

AttributeError: 'dict' object has no attribute 'estimateGas'

In [61]:
contract

web3._utils.datatypes.Contract

In [ ]:
 web3.eth.estimateGas({
     "from"      : walletbase,       
     "nonce"     : value, 
     "to"        : contractAddr,     
     "data"      : data
})

In [70]:
contract.address

'0xe7b8897D824C78e5C1076023508C43Eace3D2Afc'

In [71]:
contract.functions.rewardAddresses(ids_manageraddresses[campaign_id], int(campaign_id), rewards_pending).estimateGas(    {'gas':3000000, 
     'gasPrice': web3.toWei('23', 'gwei'),
     'from': secret['ETHBACKENDPUBLIC'],
     'to': contract_address,
     'nonce': web3.eth.getTransactionCount(secret['ETHBACKENDPUBLIC'])
    })

# https://github.com/ethereum/web3.py/blob/master/web3/contract.py

ValueError: Cannot set to in estimateGas transaction